# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transform, download=True)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
# reference https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist     
#use two 3*3 kernel replace 5*5 kernel
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1,32,kernel_size = 3),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(32),
                                   nn.Conv2d(32,32,kernel_size = 3),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(32),
                                   nn.MaxPool2d(2,2),
                                   nn.Dropout(0.4))
        self.layer2 = nn.Sequential(nn.Conv2d(32,64,kernel_size = 3),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64),
                                    nn.Conv2d(64,64,kernel_size = 3),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64),
                                    nn.MaxPool2d(2,2),
                                    nn.Dropout(0.4))
        self.fc1 = nn.Linear(1024,128)
        self.relu = nn.ReLU()
        self.drop1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(128,64)
        self.drop2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(64,10)
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x=torch.flatten(x,start_dim =1)
        x = self.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return x
    
model = SimpleNet()



    


# TODO:define loss function and optimiter
if torch.cuda.is_available():
    model = model.cuda()
    available = True
else: available = False    
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
from tqdm.notebook import tqdm
model.train()
actual_len_train = len(train_dataset)-len(train_dataset)%BATCH_SIZE
actual_len_test = len(test_dataset)-len(test_dataset)%BATCH_SIZE

for epoch in range(NUM_EPOCHS):
    total_loss = 0
    model.train()
    for images, labels in tqdm(train_loader):
        if available:
            images = images.cuda()
            labels = labels.cuda()
        pred = model(images)
        loss = nn.CrossEntropyLoss()(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    

    avg_loss = total_loss/actual_len_train
    print('loss = ',avg_loss)
    
   #eval on test_dataset      
    model.eval()
    correct = 0
    for images, labels in tqdm(test_loader):
        if available:
            images = images.cuda()
            labels = labels.cuda()
        pred = model(images)
        pred = pred.argmax(dim = 1)
        correct += torch.eq(pred,labels).sum().item()
    accuracy = correct/actual_len_test
    print('accuracy = %0.2f%%'% (accuracy*100))
    
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
 
    
    



loss =  0.002256770267179405



accuracy = 98.75%



loss =  0.0006893212402378867



accuracy = 98.97%



loss =  0.0005063819099539102


...


accuracy = 99.39%



loss =  0.0002915498959511876



accuracy = 99.39%



loss =  0.00026998629018790723



accuracy = 99.51%


In [5]:
correct = 0
for images, labels in tqdm(test_loader):
    if available:
        images = images.cuda()
        labels = labels.cuda()
    pred = model(images)
    pred = pred.argmax(dim = 1)
    correct += torch.eq(pred,labels).sum().item()
test_accuracy = correct/actual_len_test
 

correct = 0
for images, labels in tqdm(train_loader):
    if available:
        images = images.cuda()
        labels = labels.cuda()
    pred = model(images)
    pred = pred.argmax(dim = 1)
    correct += torch.eq(pred,labels).sum().item()
train_accuracy = correct/actual_len_train
print('train_accuracy = %0.2f%%'% (train_accuracy*100))  
print('test_accuracy = %0.2f%%'% (test_accuracy*100)) 


train_accuracy = 99.67%
test_accuracy = 99.51%


#### Q5:
Please print the training and testing accuracy.